In [21]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [22]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [23]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [24]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [25]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [26]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [27]:
df['loc']

1     (17901 Collins Ave #4207, Sunny Isles Beach, F...
2     (18101 Collins Ave APT 1809, Sunny Isles Beach...
3     (101 20th St #3101, Miami Beach, FL 33139, USA...
4     (102 24th St APT 1114, Miami Beach, FL 33139, ...
5     (17001 Collins Ave APT 1704, Sunny Isles Beach...
6     (789 Crandon Blvd APT 1402, Key Biscayne, FL 3...
7     (9601 Collins Ave APT 301, Bal Harbour, FL 331...
8     (4100 Island Blvd #1504, Aventura, FL 33160, U...
9     (2001 Meridian Ave APT 307, Miami Beach, FL 33...
10    (17301 Biscayne Blvd #203, North Miami Beach, ...
Name: loc, dtype: object

In [28]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [29]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Estates at Aqualina\r17901 Collins Ave 4207\rS...,6/30/2022,11/1/2022,42,"$11,000,000.00",5573,"$1,973.80",Marla Cohen,Acqualina Realty,Adriana Vargas Hernandez,Acqualina Realty,Estates at Aqualina 17901 Collins Ave 4207 Sun...,Estates at Aqualina,17901 Collins Ave 4207 Sunny Isles Beach,"(17901 Collins Ave #4207, Sunny Isles Beach, F...","(25.9420489, -80.12019839999999, 0.0)",25.942049,-80.120198,0.0
2,Trump Palace\r18101 Collins Ave 1809\rSunny Is...,9/1/2022,11/1/2022,39,"$3,000,000.00",2681,"$1,232.03",Lana Bell,Douglas Elliman,Patricia Hachar,"Compass Florida, LLC.",Trump Palace 18101 Collins Ave 1809 Sunny Isle...,Trump Palace,18101 Collins Ave 1809 Sunny Isles Beach,"(18101 Collins Ave APT 1809, Sunny Isles Beach...","(25.9440252, -80.1202751, 0.0)",25.944025,-80.120275,0.0
3,Setai\r101 20th St 3101\rMiami Beach,6/16/2022,10/31/2022,117,"$2,850,000.00",1198,"$2,378.96",Diana Garchitorena,Douglas Elliman,Devin Kay,Douglas Elliman,Setai 101 20th St 3101 Miami Beach,Setai,101 20th St 3101 Miami Beach,"(101 20th St #3101, Miami Beach, FL 33139, USA...","(25.7956082, -80.1281603, 0.0)",25.795608,-80.128160,0.0
4,1 Hotel & Homes\r102 24th St 1114\rMiami Beach,10/7/2022,10/31/2022,1,"$2,550,000.00",1084,"$2,628.87",Tracy Galya,Douglas Elliman,Tessa Trlaja,COSMORE Florida Corp.,1 Hotel & Homes 102 24th St 1114 Miami Beach,,1 Hotel & Homes 102 24th St 1114 Miami Beach,"(102 24th St APT 1114, Miami Beach, FL 33139, ...","(25.799494, -80.1268322, 0.0)",25.799494,-80.126832,0.0
5,Jade Beach\r17001 Collins Ave 1704\rSunny Isle...,10/10/2022,11/2/2022,6,"$2,450,000.00",1927,"$1,271.41",Marcello Agostini,"AG Real Estate Advisors, LLC",Marina Petlyakova,Golden Keys Inc,Jade Beach 17001 Collins Ave 1704 Sunny Isles ...,Jade Beach,17001 Collins Ave 1704 Sunny Isles Beach,"(17001 Collins Ave APT 1704, Sunny Isles Beach...","(25.9335244, -80.1211641, 0.0)",25.933524,-80.121164,0.0
6,The Ocean Club\r789 Crandon Blvd 1402\rKey Bis...,9/6/2022,10/31/2022,13,"$2,395,000.00",2100,"$1,140.48",Douglas Kinsley,Fortune Int'l Realty Key Biscayne Inc,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",The Ocean Club 789 Crandon Blvd 1402 Key Biscayne,The Ocean Club,789 Crandon Blvd 1402 Key Biscayne,"(789 Crandon Blvd APT 1402, Key Biscayne, FL 3...","(25.6868145, -80.15900479999999, 0.0)",25.686815,-80.159005,0.0
7,Majestic\r9601 Collins Ave 301\rBal Harbour,9/29/2022,11/4/2022,11,"$2,000,000.00",2260,$884.96,Shneur Shapira,The Brokerage South Florida Real Estate,Joel Lusky,The Brokerage South Florida Real Estate,Majestic 9601 Collins Ave 301 Bal Harbour,Majestic,9601 Collins Ave 301 Bal Harbour,"(9601 Collins Ave APT 301, Bal Harbour, FL 331...","(25.8876563, -80.1222319, 0.0)",25.887656,-80.122232,0.0
8,Bellini Williams Island\r4100 Island Blvd 1504...,7/25/2022,11/1/2022,77,"$1,900,000.00",2608,$728.53,Nicole Shein,"Williams Island Realty, LLC",Joel Matus,"Williams Island Realty, LLC",Bellini Williams Island 4100 Island Blvd 1504 ...,Bellini Williams Island,4100 Island Blvd 1504 Aventura,"(4100 Island Blvd #1504, Aventura, FL 33160, U...","(25.9410759, -80.1359681, 0.0)",25.941076,-80.135968,0.0
9,The Meridian Condo\r2001 Meridian Ave 307\rMia...,5/11/2022,10/31/2022,128,"$1,722,500.00",2058,$836.98,Sebastian Wagner,"Brokers, LLC",Maurice Boschetti,"Boschetti Realty Group, LLC.",The Meridian Condo 2001 Meridian Ave 307 Miami...,The Meridian Condo,2001 Meridian Ave 307 Miami Beach,"(2001 Meridian Ave APT 307, Miami Beach, FL 33...","(25.7967762, -80.1357863, 0.0)",25.796776,-80.135786,0.0
10,M0 arina Palms Residences\r17301 Biscayne Blvd...,5/13/2022,10/31/2022,158,"$1,705,000.00",2219,$768.36,Tania Weiss,Fortune International Realty,Marc Plotkin,Coldwell Banker Realty,M0 arina Palms Residences 17301 Biscayne Blvd ...,M,0 arina Palms Residences

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [30]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Estates at Aqualina\r17901 Collins Ave 4207\rS...,6/30/2022,11/1/2022,42,"$11,000,000.00",5573,"$1,973.80",Marla Cohen,Acqualina Realty,Adriana Vargas Hernandez,Acqualina Realty,Estates at Aqualina 17901 Collins Ave 4207 Sun...,Estates at Aqualina,17901 Collins Ave 4207 Sunny Isles Beach,"(17901 Collins Ave #4207, Sunny Isles Beach, F...","(25.9420489, -80.12019839999999, 0.0)",25.942049,-80.120198,0.0
2,Trump Palace\r18101 Collins Ave 1809\rSunny Is...,9/1/2022,11/1/2022,39,"$3,000,000.00",2681,"$1,232.03",Lana Bell,Douglas Elliman,Patricia Hachar,"Compass Florida, LLC.",Trump Palace 18101 Collins Ave 1809 Sunny Isle...,Trump Palace,18101 Collins Ave 1809 Sunny Isles Beach,"(18101 Collins Ave APT 1809, Sunny Isles Beach...","(25.9440252, -80.1202751, 0.0)",25.944025,-80.120275,0.0
3,Setai\r101 20th St 3101\rMiami Beach,6/16/2022,10/31/2022,117,"$2,850,000.00",1198,"$2,378.96",Diana Garchitorena,Douglas Elliman,Devin Kay,Douglas Elliman,Setai 101 20th St 3101 Miami Beach,Setai,101 20th St 3101 Miami Beach,"(101 20th St #3101, Miami Beach, FL 33139, USA...","(25.7956082, -80.1281603, 0.0)",25.795608,-80.128160,0.0
4,1 Hotel & Homes\r102 24th St 1114\rMiami Beach,10/7/2022,10/31/2022,1,"$2,550,000.00",1084,"$2,628.87",Tracy Galya,Douglas Elliman,Tessa Trlaja,COSMORE Florida Corp.,1 Hotel & Homes 102 24th St 1114 Miami Beach,,1 Hotel & Homes 102 24th St 1114 Miami Beach,"(102 24th St APT 1114, Miami Beach, FL 33139, ...","(25.799494, -80.1268322, 0.0)",25.799494,-80.126832,0.0
5,Jade Beach\r17001 Collins Ave 1704\rSunny Isle...,10/10/2022,11/2/2022,6,"$2,450,000.00",1927,"$1,271.41",Marcello Agostini,"AG Real Estate Advisors, LLC",Marina Petlyakova,Golden Keys Inc,Jade Beach 17001 Collins Ave 1704 Sunny Isles ...,Jade Beach,17001 Collins Ave 1704 Sunny Isles Beach,"(17001 Collins Ave APT 1704, Sunny Isles Beach...","(25.9335244, -80.1211641, 0.0)",25.933524,-80.121164,0.0
6,The Ocean Club\r789 Crandon Blvd 1402\rKey Bis...,9/6/2022,10/31/2022,13,"$2,395,000.00",2100,"$1,140.48",Douglas Kinsley,Fortune Int'l Realty Key Biscayne Inc,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",The Ocean Club 789 Crandon Blvd 1402 Key Biscayne,The Ocean Club,789 Crandon Blvd 1402 Key Biscayne,"(789 Crandon Blvd APT 1402, Key Biscayne, FL 3...","(25.6868145, -80.15900479999999, 0.0)",25.686815,-80.159005,0.0
7,Majestic\r9601 Collins Ave 301\rBal Harbour,9/29/2022,11/4/2022,11,"$2,000,000.00",2260,$884.96,Shneur Shapira,The Brokerage South Florida Real Estate,Joel Lusky,The Brokerage South Florida Real Estate,Majestic 9601 Collins Ave 301 Bal Harbour,Majestic,9601 Collins Ave 301 Bal Harbour,"(9601 Collins Ave APT 301, Bal Harbour, FL 331...","(25.8876563, -80.1222319, 0.0)",25.887656,-80.122232,0.0
8,Bellini Williams Island\r4100 Island Blvd 1504...,7/25/2022,11/1/2022,77,"$1,900,000.00",2608,$728.53,Nicole Shein,"Williams Island Realty, LLC",Joel Matus,"Williams Island Realty, LLC",Bellini Williams Island 4100 Island Blvd 1504 ...,Bellini Williams Island,4100 Island Blvd 1504 Aventura,"(4100 Island Blvd #1504, Aventura, FL 33160, U...","(25.9410759, -80.1359681, 0.0)",25.941076,-80.135968,0.0
9,The Meridian Condo\r2001 Meridian Ave 307\rMia...,5/11/2022,10/31/2022,128,"$1,722,500.00",2058,$836.98,Sebastian Wagner,"Brokers, LLC",Maurice Boschetti,"Boschetti Realty Group, LLC.",The Meridian Condo 2001 Meridian Ave 307 Miami...,The Meridian Condo,2001 Meridian Ave 307 Miami Beach,"(2001 Meridian Ave APT 307, Miami Beach, FL 33...","(25.7967762, -80.1357863, 0.0)",25.796776,-80.135786,0.0
10,M0 arina Palms Residences\r17301 Biscayne Blvd...,5/13/2022,10/31/2022,158,"$1,705,000.00",2219,$768.36,Tania Weiss,Fortune International Realty,Marc Plotkin,Coldwell Banker Realty,M0 arina Palms Residences 17301 Biscayne Blvd ...,M,0 arina Palms Residences

In [37]:
df.at[1,'building_name']=('Estates at Acqualina')

df.at[4,'building_name']=('1 Hotel & Homes')
df.at[4,'address_only']=('24th St 1114')

df.at[10,'building_name']=('Brickell Flatiron')
df.at[10,'address_only']=('17301 Biscayne Blvd')

In [32]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [38]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('October 30th - November 5th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [39]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [35]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [36]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_110722
